# Manually modelling helicity amplitudes

The goal in this notebook is to formulate helicity amplitude model manually and reproduce the result in [Amplitude model with `ampform-dpd` notebook](ampform-dpd.ipynb).

In [ ]:
from __future__ import annotations

import logging
import os
import warnings

STATIC_PAGE = "EXECUTE_NB" in os.environ

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.disable(logging.WARNING)
warnings.filterwarnings("ignore")

## Model implementation